In [ ]:
import random
import spacy
import wikipedia
import telegram
import math
from random import *

from clarifai.rest import ClarifaiApp, Image
from spellchecker import SpellChecker
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

wiki_api = wikipediaapi.Wikipedia('en')
bot = telegram.Bot(token='<token>')

### Read and Initialize Information

In [ ]:
mathematicians = pd.read_csv('mathematicians2.csv') 
math_words = pd.read_csv('words.csv') 

lastname = list(mathematicians['lastname'])
mathematicians["fullname"] = mathematicians["firstname"] + mathematicians["lastname"]
fullname = list(mathematicians["fullname"])
math_words_list = list(math_words["word"])
math_words.loc[math_words['word']=='bell curve']

shapes = {
    'square':{
        'description': "A Square is a flat shape with 4 equal sides and every angle is a right angle (90°).",
    },
    'rectangle':{
        'description': "A rectangle is a four-sided flat shape where every angle is a right angle (90°).",
    },'triangle':{
        'description': "A triangle has three sides and three angles.",
    }, 'parallelogram':{
        'description': "A Parallelogram is a flat shape with opposite sides parallel and equal in length.",
    },'trapezoid':{
        'description': "A trapezoid is a 4-sided flat shape with straight sides that has a pair of opposite sides parallel ",
    }, 'cube':{
        'description': "A cube is a three-dimensional solid object bounded by six square faces, facets or sides, with three meeting at each vertex. ",
    },'cylinder':{
        'description': "A cylinder is a closed solid that has two parallel (usually circular) bases connected by a curved surface.",
    },'cone':{
        'description': "A cone is a solid that has a circular base and a a single vertex.",
    },'sphere':{
        'description': "A sphere is a geometrical object in three-dimensional space that is the surface of a ball.",
    }
}

concepts = ['perimeter', 'area', 'volume']

formula = {
    'perimeter': {
        'square': 'Formula = 4 × a, a = length of side',
        'rectangle': 'Formula = 2 × (a + b)',
        'triangle': 'Formula = a + b + c',
        'circle': 'Formula = 2πr, r = radius'
    },
    'area':{
        'square': 'Formula = a2, a = length of side',
        'rectangle': 'Formula = w × h, w = width, h = height',
        'circle': 'Formula = π × r2, Circumference = 2 × π × r, r = radius',
        'parallelogram': 'Formula = b × h, b = base, h = vertical height',
        'trapezoid': 'Formula = ½(a+b) × h, h = vertical height'
    },
    'volume':{
        'triangle': 'Formula: 1/2 x b x h, b is the length of the base and h is the length of the height',
        'cube': 'Formula: s x 3, s is the length of the side.',
        'cyclinder': 'Formula: pi x r x 2 x h, L is the length, W is the width and H is the height.',
        'cone': 'Formula: 1/3 x pi x r x 2 x h, r is the radius, h is the height.',
        'sphere': 'Formula: 4/3 x pi x r x 3, r is the radius.'
    }
}

### Welcome Message

In [ ]:
def start(update, context):
    global username, reset_chat
    username = ''
    reset_chat = True
    
    update.message.reply_text('Welcome! I am Matt the Mutt, your friendly furry companion to guide you in your PAWesome adventure in the world of Mathematics. What’s your name?')

### Process Keyphrase

In [ ]:
def keyphrase(doc, update):
    if len(doc) == 1:
        if doc[0].text in lastname:
            return {'result': mathematicians.loc[mathematicians['lastname'] == str(doc[0].text)]['summary'].values}
        elif (doc[0].dep_== 'ROOT' and doc[0].pos_ == 'NOUN') or (doc[0].dep_== 'ROOT' and doc[0].pos_ == 'PROPN'):
            return doc[0].lemma_.capitalize()
    else:
        if doc[0].pos_ == 'PRON':
            pronoun_list = []
            noun_list = []
            available_shape = ''
            available_concept = ''
            for t in reversed(doc):
                if t.text in shapes.keys():
                    available_shape = str(t.text.lower())
                if t.text in concepts:
                    available_concept = str(t.text.lower()) 
                    
                if t.pos_ == 'PROPN' or t.text in lastname:
                    pronoun_list.append(t.text)
                elif t.pos_ == 'NOUN':
                    noun_list.append(t.lemma_.lower())    
                else:
                    if t.lemma_ in math_words_list:
                        return {'result': math_words.loc[math_words['word'] == t.lemma_]['description'].values}
                    else:
                        if t.i < len(doc)-1:
                            index_value = int(t.i)
                            check_word = '{} {}'.format(str(doc[t.i+1]), str(doc[index_value]))
                            if check_word in math_words_list:
                                return {'result': math_words.loc[math_words['word'] == check_word]['description'].values}                                
                
            
            if available_concept != '' and available_shape != '':
                if available_concept in formula.keys():
                    if available_shape in formula[available_concept].keys():
                        msg_id = update.message.chat

                        bot.send_photo(msg_id['id'], open('formula/{}_{}.png'.format(available_concept, available_shape), 'rb'))

                        return {'result': formula[available_concept][available_shape]}
                    else:
                        return {'result': 'Sorry, the {} of {} formula is not yet available'.format(available_concept, available_shape)}
                else:
                    return {'result': 'Sorry, the {} of {} formula is not yet available'.format(available_concept, available_shape)}
            
            elif available_shape != '':
                msg_id = update.message.chat
                bot.send_photo(msg_id['id'], open('shapes/{}.png'.format(available_shape), 'rb'))
                return {'result': shapes[available_shape]['description']}
                
                
            if len(pronoun_list) > 1:
                pronoun_list.reverse()
                pronoun_list = ' '.join(pronoun_list)
                
                if pronoun_list in fullname:
                    return {'result': mathematicians.loc[mathematicians['fullname'] == pronoun_list]['summary'].values}
                else:
                    return pronoun_list
            elif len(pronoun_list) == 1:
                if pronoun_list[0] in lastname:
                       return {'result': mathematicians.loc[mathematicians['lastname'] == pronoun_list[0]]['summary'].values}
            else:
                if len(noun_list) > 0:
                    return ' '.join(noun_list)
            
        for t in doc:
            if t.dep_ == 'pobj' and (t.pos_ == 'NOUN' or t.pos_ == 'PROPN'):
                return (' '.join([child.text for child in t.lefts]) + ' ' + t.text).lstrip()
            
            for t in reversed(doc):
                if t.dep_ == 'nsubj' and (t.pos_ == 'NOUN' or t.pos_ == 'PROPN'):
                    return t.text + ' ' + t.head.text
                
            for t in reversed(doc):
                if t.dep_ == 'dobj' and (t.pos_ == 'NOUN' or t.pos_ == 'PROPN'):
                    return t.head.text + 'ing' + ' ' + t.text
        return False

### Wikipedia Checker

In [ ]:
def wiki(concept, msg):
    nlp = spacy.load('en')
    doc_concept = nlp(concept)
    concept = ' '.join(map(str, list(doc_concept.noun_chunks)))
    
    wiki_resp = wiki_api.page(concept)
    r1 = re.search("\(([^)]+)\)", wiki_resp.summary)
    summary = wiki_resp.summary
    summary = summary.split("\n")
    summary = summary[0]
    result = wiki_resp.summary
    if r1 is not None:
        span = r1.span()
        result = wiki_resp.summary.replace(str(summary[span[0]:span[1]]), "")
    
    result = result.split(".")    
    if len(result) > 0:
        doc = nlp(result[0])
        for sent in doc.sents:
            for t in sent:
                if t.text == concept and t.dep_ == 'dobj':
                    return sent.text
        if len(list(doc.sents)) > 0: 
            return list(doc.sents)[0].text
        else:
            return 'No results found: Try Searching {}'.format(check_spelling(msg))
    else:
        return 'No results found: Possible Suggestion {}'.format(check_spelling(msg))

### Spelling Checker

In [ ]:
def check_spelling(msg):
    result = ""
    spell = SpellChecker()
    
    msg_list = msg.split(" ")
    misspelled = spell.unknown(msg_list)
    misspelled = list(misspelled)
    
    for _msg_list in msg_list:
        if _msg_list in misspelled:
            result += spell.correction(_msg_list)+" "
        else:
            result += _msg_list+" "
            
    return result.strip()

### Text Checker Which Selection to Go

In [ ]:
def text_msg(update, context):
    msg = update.message.text
    global reset_chat, answer, current_mode
    if reset_chat:
        global username
        username = msg
        reset = False
        message = """Hi {}, it is very nice to meet you. I hope we can be friends for a long time. Let’s start by choosing the mode:
        \n
        /search - Search the web with important math concepts and famous mathematicians.
         \n
        /compute - Compute simple problems in Algebra, Geometry and Trigonometry.
         \n
        /play - Review by answering some questions based on the topics we reviewed.
         \n
        /help - Read the documentation and instructions in using this chatbot.
        \n
        /quit - Exit chatbot""".format(username)

        update.message.reply_text(message)
    
    else:
        reset_chat = False
        if current_mode == 'search':
            nlp = spacy.load('en')
            doc = nlp(msg)
            concept = keyphrase(doc, update)
            if type(concept) == dict:
                if type(concept['result']) == 'list':
                    update.message.reply_text(concept['result'][0])
                else:
                    update.message.reply_text(concept['result'])
            else:
                if concept != False:
                    update.message.reply_text(wiki(concept, msg))
                else:
                    update.message.reply_text('Please rephrase your question.')
        elif current_mode == 'compute':
            num = update.message.text
            num = num.strip()
            compute = ''
            try:
                compute = "{}".format(eval(num))
            except:
                compute = 'Please input a valid arithmetic operation.'
            update.message.reply_text(compute)
        reset_chat = False
        if current_mode == 'Waiting for answer...':
                answer_response = update.message.text
                try:
                    answer_response = int(answer_response)
                    if answer_response == answer:
                        current_mode = "review"
                        update.message.reply_text("You're PAWesome!")
                    else:
                        update.message.reply_text("Try again. Be PAWsitive, you can do it!")
                except:
                    update.message.reply_text("Input a number.")

### Start Message for Search

In [ ]:
def search(update, context):
    global current_mode, reset_chat
    current_mode = 'search'
    reset_chat = False
    update.message.reply_text("""Hi {}, want to know some mathematical concepts or famous mathematicians? Try typing ‘Who is Archimedes?’ to start!""".format(username))

### Start Message for Compute

In [ ]:
def compute(update, context):
    global current_mode, reset_chat
    current_mode = 'compute'
    reset_chat = False
    update.message.reply_text("""Hi {}, let's compute some arithmetic and trigonometric problems. Are you ready because I sure am. Let's go
        \n
        Arithmetic - ex. (((5+1)-(1+1))*factorial(2))/ remainder(4,3)
        \n
        Trigonometric - ex. sin(pi/2), degrees(acos(0.85))""".format(username))

### Start Message for Play

In [ ]:
def play(update, context):
    global current_mode, reset_chat
    current_mode = 'games'
    reset_chat = False
    update.message.reply_text("""Hi {}, now that we've learned a lot. Let us try solving some problems. Let us bring your knowledge to the test. Let's start by choosing a difficulty level:
        \n
        /easy - Solve simple arithmetic problems!
        \n
        /average - Let's try harder problems!
        \n
        /difficult - That's great! I like the confidence!""".format(username))

### Easy Math Questions

In [ ]:
def easy(update, context):
    global current_mode, reset_chat, score
    current_mode = 'easy'
    reset_chat = False
    math_easy(update, context)

In [ ]:
def math_easy(update, context):
    global answer, current_mode
    num1 = randint(1, 10)
    num2 = randint(1, 10)
    symbol = randint(1,3)
    
    if symbol == 1:
        question = ("What is " + str(num1) + "+" + str(num2) + "?")
        answer = num1 + num2
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)

    elif symbol == 2:
        question = ("What is " + str(num1) + "-" + str(num2) + "?")
        answer = num1 - num2
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)

    elif symbol == 3:
        question = ("What is " + str(num1) + "*" + str(num2) + "?")
        answer = num1 * num2
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)

### Average Math Questions

In [ ]:
def average(update, context):
    global current_mode, reset_chat, score
    current_mode = 'average'
    reset_chat = False
    math_average(update, context)

In [ ]:
def math_average(update, context):
    global answer, current_mode
    num1 = randint(1, 10)
    num2 = randint(1, 10)
    num3 = randint(1, 5)
    symbol = randint(1, 3)
    
    if symbol == 1:
        question = ("What is " + str(num1) + "+" + str(num2) + "+" + str(num3) + "?")
        answer = num1 + num2 + num3
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)        

    elif symbol == 2:
        question = ("What is " + str(num1) + "-" + str(num2) + "-" + str(num3) + "?")
        answer = num1 - num2 - num3
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)
        
    elif symbol == 3:
        question = ("What is " + str(num1) + "*" + str(num2) + "*" + str(num3) +"?")
        answer = num1 * num2 * num3
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)

### Difficult Math Questions

In [ ]:
def difficult(update, context):
    global current_mode, reset_chat, score
    current_mode = 'difficult'
    reset_chat = False
    math_difficult(update, context)

In [ ]:
def math_difficult(update, context):
    global answer, current_mode
    symbol = randint(1, 2)
    
    if symbol == 1:
        num1 =randint(1,10)
        question = ("What is " + str(num1) + " factorial ?")
        answer = factorial(num1)
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)     

    elif symbol == 2:
        list_number = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900]
        num2 = list_number[randint(0,30)]
        question = ("What is the square root of " + str(num2) + " ?")
        answer = sqrt(num2)
        current_mode = 'Waiting for answer...'
        update.message.reply_text(question)     

### Show Documentation

In [ ]:
def show_help(update, context):
    global current_mode, reset_chat
    current_mode = 'help'
    reset_chat = False
    update.message.reply_text("""Hi {}, Please refer to the following links for the documentation:""".format(username))

### Quit Chatbot

In [ ]:
def show_quit(update, context):
    global current_mode, reset_chat
    current_mode = 'quit'
    reset_chat = True
    update.message.reply_text("""Hi {}, Have a nice day and goodbye""".format(username))

### Main Application

In [ ]:
def main():
    updater = Updater("<token>", use_context=True)
    disp = updater.dispatcher
    disp.add_handler(CommandHandler("start", start))
    disp.add_handler(CommandHandler("search", search))
    disp.add_handler(CommandHandler("compute", compute))
    disp.add_handler(CommandHandler("play", play))
    disp.add_handler(CommandHandler("easy", easy))
    disp.add_handler(CommandHandler("average", average))
    disp.add_handler(CommandHandler("difficult", difficult))
    disp.add_handler(CommandHandler("help", show_help))
    disp.add_handler(CommandHandler("quit", show_quit))
    disp.add_handler(MessageHandler(Filters.text, text_msg))
    updater.start_polling()
    updater.idle()

In [ ]:
if __name__ == '__main__':
    main()